In [ ]:
import os

os.environ[
    "OMP_NUM_THREADS"
] = "64"  # for jupyter.nersc.gov otherwise the notebook only uses 2 cores

# Preprocess HASLAM map as it is done for Synchrotron Amplitude template

In [ ]:
from pathlib import Path
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np

# import pymaster as nmt
from astropy.io import fits

%matplotlib inline

In [ ]:
plt.style.use("seaborn-talk")

In [ ]:
import pysm3 as pysm
import pysm3.units as u

nside = 512
lmax = 3 * nside
comp = "IQU"
components = list(enumerate(comp))
components

In [ ]:
datadir = Path("/global/cscratch1/sd/giuspugl/extending_synch/data")
imapfile = datadir / "haslam408_dsds_Remazeilles2014.fits"
qumapfile = datadir / "wmap_band_iqumap_r9_9yr_K_v5.fits"
if not imapfile.exists():
    !wget -O $imapfile   https://lambda.gsfc.nasa.gov/data/foregrounds/haslam_2014/haslam408_dsds_Remazeilles2014.fits
if not qumapfile.exists():
    !wget -O $qumapfile   https://lambda.gsfc.nasa.gov/data/map/dr5/skymaps/9yr/raw/wmap_band_iqumap_r9_9yr_K_v5.fits
imap = hp.read_map(imapfile)
qumap = hp.read_map(qumapfile, field=[1, 2])

In [ ]:
rescaling_factor = (23.0 / 0.408) ** -3.1

imap *= rescaling_factor
imap <<= u.K_RJ
imap = imap.to(u.uK_RJ)
qumap <<= u.mK_RJ
qumap = qumap.to("uK_RJ")
IQU = np.array([imap, qumap[0], qumap[1]])
FWHM_SMOOTHING = 2  # deg
IQU = hp.smoothing(IQU, fwhm=np.radians(FWHM_SMOOTHING))
IQU <<= u.uK_RJ
plt.figure(figsize=(20, 5))
for i_pol, pol in components:
    hp.mollview(
        IQU[i_pol],
        title="Haslam/WMAP K synch " + pol,
        sub=131 + i_pol,
        unit=IQU.unit,
        min=-300,
        max=300,
    )

In [ ]:
import numpy as np
import healpy as hp


def map_to_log_pol_tens(m):
    P = np.sqrt(m[1] ** 2 + m[2] ** 2)
    log_pol_tens = np.empty_like(m)
    log_pol_tens[0] = np.log(m[0] ** 2 - P ** 2) / 2.0
    log_pol_tens[1:] = m[1:] / P * np.log((m[0] + P) / (m[0] - P)) / 2.0
    return log_pol_tens


def log_pol_tens_to_map(log_pol_tens):
    P = np.sqrt(log_pol_tens[1] ** 2 + log_pol_tens[2] ** 2)
    m = np.empty_like(log_pol_tens)
    exp_i = np.exp(log_pol_tens[0])
    m[0] = exp_i * np.cosh(P)
    m[1:] = log_pol_tens[1:] / P * exp_i * np.sinh(P)
    return m


def sigmoid(x, x0, width, power=4):
    """Sigmoid function given start point and width
    Parameters
    ----------
    x : array
        input x axis
    x0 : float
        value of x where the sigmoid starts (not the center)
    width : float
        width of the transition region in unit of x
    power : float
        tweak the steepness of the curve
    Returns
    -------
    sigmoid : array
        sigmoid, same length of x"""
    return 1.0 / (1 + np.exp(-power * (x - x0 - width / 2) / width))


iqu = map_to_log_pol_tens(IQU.value)

In [ ]:
print(
    f"{np.isnan(iqu[0]).sum()  } pixels out of { iqu[0].size} are NaNs in Log Pol Tens maps "
)
for i in range(3):
    iqu[i, np.isnan(iqu[i])] = np.nanmedian(iqu[i])
assert np.isnan(iqu).sum() == 0
print(
    f"{np.isnan(iqu[0]).sum()  } pixels out of { iqu[0].size} are NaNs in Log Pol Tens maps "
)

In [ ]:
for i_pol, pol in components:
    hp.mollview(iqu[i_pol], title="Log Pol tensor " + pol, sub=131 + i_pol)

### Template for $C_s$ is the $i$ map smoothed at 5deg 

The modulation amplitude is exactly the same as the one adopted for $Bs$  and $intensity $ small scales 

In [ ]:
ismooth = hp.smoothing(iqu[0], fwhm=np.radians(5))
minmax = lambda m, a, b: a + (b - a) * (m - m.min()) / (m.max() - m.min())

modulate_amp = (ismooth) * 1.0

mskmd = ismooth > 5.5
b1 = 1.5
a = 0.5

mskmd = ismooth > 5

modulate_amp[mskmd] = minmax(ismooth[mskmd], 1.1, 2)
modulate_amp[~mskmd] = minmax(ismooth[~mskmd], 0.1, 1.1)

### Tweak the values for MinMax rescaling so that we match Kogut 2012 measurements within ARCADE patch 

In [ ]:
Cmap = minmax(ismooth, a=-0.041, b=-0.076)

### Load ARCADE patch 
make wget link 

In [ ]:
arcadefile = datadir / "arc2_3150_v19.fits"
if not arcadefile.exists():
    !wget -O $arcadefile   https://lambda.gsfc.nasa.gov/data/suborbital/ARCADE/arc2_3150_v19.fits

arcade = hp.read_map(arcadefile)
mask = arcade != 0
mask_arcade = hp.ud_grade(mask, nside_out=hp.get_nside(Cmap))
hp.mollview(mask_arcade)

In [ ]:
print(
    f" Kogut 2012 measured Cs = -0.052\pm 0.005, we get within ARCADE patch:\n"
    + f"{Cmap[mask_arcade].mean():.4f}\pm { Cmap[mask_arcade] .std():.4f}"
)

In [ ]:
hp.gnomview(
    (Cmap * mask_arcade) / mask_arcade,
    title="Matched synchrotron curvature - ARCADE",
    rot=[66, 0],
    reso=10,
    xsize=780,
    cmap="jet",
    min=-0.1,
    max=0,
)

hp.mollview(Cmap, title="synchrotron curvature", cmap="jet", min=-0.1, max=0)
hp.graticule()

# Inject  Modulated small scales to  $C_s$ 

In [ ]:
def sigmoid(x, x0, width, power=4):
    """Sigmoid function given start point and width
    Parameters
    ----------
    x : array
        input x axis
    x0 : float
        value of x where the sigmoid starts (not the center)
    width : float
        width of the transition region in unit of x
    power : float
        tweak the steepness of the curve
    Returns
    -------
    sigmoid : array
        sigmoid, same length of x"""
    return 1.0 / (1 + np.exp(-power * (x - x0 - width / 2) / width))

In [ ]:
ell = np.arange(lmax + 1)
cl_spass_fit = 10 ** (-0.82655497) * ell ** (-2.60860208)
cl_spass_fit[0] = 0
ell_fit_low = 10
ell_fit_high = 36

In [ ]:
def run_anafast(m, lmax):
    clanaf = hp.anafast(m, lmax=lmax)
    cl = {}
    cl["TT"] = clanaf
    ell = np.arange(lmax + 1)

    cl_norm = ell * (ell + 1) / np.pi / 2
    cl_norm[0] = 1
    return ell, cl_norm, cl


def model(ell, A, gamma):
    out = A * ell ** gamma
    out[:2] = 0
    return out

In [ ]:
ell, cl_norm, cl_c = run_anafast(Cmap, lmax=lmax)

In [ ]:
gamma_spass = -2.60860208
A0 = cl_c["TT"][ell_fit_high] / ell_fit_high ** (gamma_spass)


plt.loglog(ell, cl_c["TT"], label="Anafast $C_\ell$")
plt.plot(
    ell[ell_fit_high:],
    model(ell[ell_fit_high:], A0, gamma_spass),
    label="power law SPASS ",
)

plt.axvline(
    ell_fit_low, linestyle="--", color="black", label="$ \ell={} $".format(ell_fit_low)
)
plt.axvline(
    ell_fit_high, linestyle="--", color="gray", label="$ \ell={} $".format(ell_fit_high)
)
plt.grid()
plt.title(r"$C_s  $" + f" power spectrum ")
plt.legend()
plt.ylabel("$ C_\ell [\mu K_{RJ}^2]$")
plt.xlabel(("$\ell$"))
plt.xlim(2, lmax)

In [ ]:
sig_func = sigmoid(ell, x0=ell_fit_high, width=20)
np.random.seed(777)
alm_Cs = hp.map2alm(Cmap, lmax=lmax, use_pixel_weights=True)
Cs_ls_alm = hp.almxfl(alm_Cs, np.sqrt(1.0 - sig_func))
Cs_ss_alm = hp.synalm(model(ell, A0, gamma_spass), lmax=lmax)
Cs_ss_alm = hp.almxfl(Cs_ss_alm, np.sqrt(sig_func))
Cs_ss = hp.alm2map(Cs_ss_alm, lmax=lmax, nside=nside)
Cs_ss *= modulate_amp

Cs_ls = hp.alm2map(Cs_ls_alm, nside=nside)
Cs_out = Cs_ss + Cs_ls
ell, _, clcs_out = run_anafast(Cs_out, lmax=lmax)
ell, _, clls = run_anafast(Cs_ls, lmax=lmax)
ell, _, clss = run_anafast(Cs_ss, lmax=lmax)

In [ ]:
for ii in range(1):
    plt.loglog(
        ell, cl_c["TT"], alpha=0.5, color="C%d" % ii, label=r"$ C_s$ w/o small scales"
    )
    plt.loglog(
        ell,
        clcs_out["TT"],
        label=r"$  C_s$ w/ small scales ",
    )
    # plt.loglog(ell,  clls[pol], label=r"$  \beta_s$ PySM3 ",  )
    # plt.loglog(ell,  clss[pol], label=r"$  \beta_s$ PySM3 ",  )

    plt.grid(True)

    plt.legend()

    plt.ylabel("$ C_\ell $")
    plt.xlabel(("$\ell$"))
    # plt.xlim(10, 55 )
    plt.xlim(2, lmax)
plt.ylim(1e-15, 1e-4)

In [ ]:
hp.gnomview(
    (Cmap * mask_arcade) / mask_arcade,
    title="Cs w/o small scales - ARCADE",
    rot=[66, 0],
    reso=10,
    sub=121,
    xsize=780,
    cmap="jet",
    min=-0.1,
    max=0,
)
hp.gnomview(
    (Cs_out * mask_arcade) / mask_arcade,
    title="Cs w/ small scales - ARCADE",
    rot=[66, 0],
    reso=10,
    sub=122,
    xsize=780,
    cmap="jet",
    min=-0.1,
    max=0,
)
plt.figure()
hp.mollview(Cmap, title="Cs w/o small scales", cmap="jet", min=-0.1, max=0, sub=121)


hp.mollview(Cs_out, title="Cs w/ small scales", cmap="jet", min=-0.1, max=0, sub=122)
hp.graticule()

In [ ]:
print(
    f" Kogut 2012 measured Cs = -0.052\pm 0.005, we get within ARCADE patch:\n"
    + f"{Cs_out[mask_arcade].mean().mean():.4f}\pm { Cs_out[mask_arcade]  .std():.4f}"
)